## Imports

In [1]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from importlib.metadata import requires, PackageNotFoundError

## Functions

In [2]:
def get_strict_dependencies(package_name):
    try:
        dependencies = requires(package_name)
        if dependencies is None:
            return []

        # Extract just the package names, ignoring versions and conditions
        strict_dependencies = [dep.split(';', 1)[0].split('>=')[0].split('==')[0].strip() for dep in dependencies if ';' not in dep]
        return strict_dependencies

    except PackageNotFoundError:
        print(f"Package {package_name} not found.")
        return []


In [3]:

def is_download_finished(directory):
    # Wait for the download to finish (no .tmp and .crdownload files)
    while any([filename.endswith('.tmp') or filename.endswith('.crdownload') for filename in os.listdir(directory)]):
        time.sleep(1)
    return True


In [4]:
def download_whl_files(package, driver, directory):
    driver.get(f"https://pypi.org/project/{package}/#files")
    time.sleep(3)  # Wait for the page to load

    def download_and_wait(file_type):
        xpath_expression = f"//a[contains(@href, '{file_type}')]"
        if file_type == 'py3-none-any.whl':
            xpath_expression = f"//a[contains(@href, 'py3-none-any.whl') and not(contains(@href, 'win_amd64'))]"

        files = driver.find_elements(By.XPATH, xpath_expression)
        for file in files:
            file_url = file.get_attribute('href')
            file_name = file_url.split('/')[-1]

            if not os.path.exists(os.path.join(directory, file_name)):
                print(f"Downloading {file_type} file for {package} from: {file_url}")
                driver.get(file_url)
                is_download_finished(directory)
            else:
                print(f"File {file_name} already exists. Skipping download.")

    download_and_wait(f"{python_version}-win_amd64.whl")
    download_and_wait('py3-none-any.whl')


## Inputs for Python Package Downloader Script

This script requires specific inputs to function correctly. Ensure you provide the following information before running the script:

1. **Python Version of Your Project**
   - Specify the Python version used in your project. 
   - This is necessary to download the correct `.whl` files compatible with your Python version.
   - **Example**: `"cp311"` for Python 3.11.



2. **Local Folder for `.whl` Files**
   - Provide the path to the local directory where you want to save the downloaded `.whl` files.
   - Ensure the script has write access to this folder.
   - **Example**: `r"C:\Users\benysar\testing_data\packages_stroke_occlusion"`



3. **List of Python Packages and Their Dependencies**
   - Specify the Python packages you need to download, including any dependencies. Use the function `get_strict_dependencies` to easily access to the list of dependencies
   - The script will attempt to download `.whl` files for each of these packages.
   - **Example**: `["numpy", "pandas"]`


### Looking for dependencies

In [5]:
package_name = "traitlets"
print(f"Strict dependencies for {package_name}: {get_strict_dependencies(package_name)}")


Strict dependencies for traitlets: []


In [7]:
python_version = "cp311"
dossier_local ="C:\\Users\\benysar\\Desktop\Github\\stroke-occlusion\\packages"
packages_to_download = ["ipykernel", 'comm','traitlets', 'debugpy', 
'ipython', 'backcall', 'decorator', 'jedi', 'parso','matplotlib-inline', 'pickleshare', 'prompt-toolkit', 'wcwidth', 'pygments', 'stack-data', 'executing', 
'asttokens', 'six', 'pure-eval', 'traitlets',
'jupyter-client', 'jupyter-core', 'python-dateutil', 'pyzmq', 'tornado', 'traitlets', 'matplotlib-inline', 
'nest-asyncio', 'packaging', 'psutil', 'pyzmq', 'tornado', 'traitlets']


In [8]:
# Configure Edge for Selenium with automatic download settings
options = Options()
options.use_chromium = True
prefs = {
    "download.default_directory": os.path.abspath(dossier_local),
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
options.add_experimental_option("prefs", prefs)

# Initialize the Edge driver
edge_service = Service('C:/Users/benysar/Documents/edgedriver_win64/msedgedriver.exe')  
driver = webdriver.Edge(service=edge_service, options=options)

# Example usage
for package in list(set(packages_to_download)):
    download_whl_files(package, driver, dossier_local)
driver.quit()